# Tutorial: Ground Penetrating Radar (GPR) — EM wave propagation and 2D FDTD modeling

This notebook explains the basic theory behind GPR (i.e. electromagnetic high-frequency propagation in the subsurface) and provides a simple 2D example producing a synthetic radargram (common‑shot gather).



## 1. Introduction

Ground Penetrating Radar (GPR) transmits short electromagnetic pulses and records the fields scattered and reflected by dielectric contrasts in the subsurface. Typical frequencies range from tens of MHz to a few GHz: higher frequency gives better resolution but less penetration.

Raw data are usually displayed as radargrams (time vs horizontal position) or imaged using migration algorithms. The goal of GPR data processing is to transform noisy, uncorrected field data into a clean and interpretable subsurface image that accurately represents reflection geometry and depth.  

<img src=../figures/4.png  style="width:33%;"/>  


### Key EM theory

Maxwell's equations (differential form, SI units) in a linear, isotropic medium with conductivity $\sigma$:

$$
\nabla \times \mathbf{E} = -\mu \frac{\partial \mathbf{H}}{\partial t},\qquad
\nabla \times \mathbf{H} = \sigma\mathbf{E} + \varepsilon \frac{\partial \mathbf{E}}{\partial t}.
$$

Combining these leads to a damped wave equation for the electric field (one polarization):

$$
\nabla^2 \mathbf{E} - \mu\varepsilon\frac{\partial^2 \mathbf{E}}{\partial t^2} - \mu\sigma\frac{\partial \mathbf{E}}{\partial t} = 0.
$$

If losses are small ($\sigma\approx 0$), the phase velocity is approximately

$$
v = \frac{1}{\sqrt{\mu\varepsilon}} = \frac{c}{\sqrt{\varepsilon_r}} \quad\text{with } \varepsilon = \varepsilon_0\varepsilon_r.
$$

**Skin depth** (for harmonic fields) in a conductive medium: $\delta = \sqrt{2/(\omega\mu\sigma)}$. For GPR frequencies and typical ground conductivities, this gives a sense of attenuation.


### Reflection coefficient and depth conversion

At normal incidence the reflection coefficient between medium 1 and medium 2 is (using intrinsic impedance)

$$
R = \frac{Z_2 - Z_1}{Z_2 + Z_1},\qquad Z = \sqrt{\frac{\mu}{\varepsilon}}.
$$

If $\mu\approx\mu_0$ then
$$
R = \frac{\sqrt{\varepsilon_{r2}} - \sqrt{\varepsilon_{r1}}}{\sqrt{\varepsilon_{r2}} + \sqrt{\varepsilon_{r1}}}
$$

Two-way travel time $t_{2w}$ for a horizontal interface at depth $z$:

$$
t_{2w} = 2 \frac{z}{v} = 2 z \sqrt{\varepsilon_r}/c.
$$

Therefore depth conversion: $z = c t_{2w} /(2\sqrt{\varepsilon_r})$ using an estimate of $\varepsilon_r$.

## 2. Simulation of GPR Bscan 2D

To simulate how GPR waves propagate, we use the Finite-Difference Time-Domain (FDTD) method — a straightforward way to solve Maxwell’s equations step by step in space and time. The implementation is intentionally simple for educational purposes.  
We start creating a simple 2d model like this:

| Zone / Object | Depth range (m) |  Description |
|----------------|----------------|--------------|
| Upper layer | 0–60 | Moderately dry soil (background) |
| Lower layer | 50–80 | Wetter, more conductive zone |
| Object 1 & 2 |   | Shallow anomaly (pipe) |

<img src=../figures/1.png  style="width:35%;"/>

We discretize the model into a regular 2D grid. We’ll use a Ricker pulse as the source (a short, band-limited impulse) and apply simple absorbing boundaries to reduce artificial reflections at the model edges. This setup produces a realistic synthetic radargram showing reflections from dielectric contrasts.  

<img src=../figures/2.png  style="width:30%;"/>  

At each grid cell, we compute the electric field (the transmitted pulse) and the magnetic field that accompanies it. These fields are updated alternately at every time step, following difference equations that approximate Maxwell’s curl relations.  

<img src=../figures/3.png  style="width:70%;"/>  


This approach allows us to “watch” how an electromagnetic pulse moves through layers of different permittivity, reflects at boundaries, and attenuates in conductive zones.

<img src=../figures/5.png  style="width:70%;"/>  


## 3. Basic Processing Steps of a Raw GPR Radargram

GPR data acquired in the field typically consists of raw traces recorded as the antenna moves along a profile. Each trace represents the amplitude of the reflected electromagnetic wave as a function of two-way travel time. Before interpretation, the raw radargram must undergo several processing steps to enhance signal quality, remove noise, and convert time into depth.

### 1. Time-Zero Correction
Aligns the start of each trace so that the first arrival (direct wave) appears at time zero.  
This compensates for small trigger delays and ensures consistent timing across traces.

### 2. Dewow Filtering
Removes low-frequency components caused by the instrument’s electronics or coupling effects.  
A high-pass or running-average filter is typically applied to restore the signal’s oscillatory character.

### 3. Background Removal
Subtracts the average trace or a low-rank component from the dataset to suppress horizontal banding and system ringing.  
This enhances the visibility of localized reflections due to subsurface targets.

### 4. Gain Application
Compensates for signal attenuation with depth by amplifying the later parts of each trace.  
Common approaches include exponential, automatic gain control (AGC), or user-defined gains.

### 5. Bandpass Filtering
Applies a frequency filter to retain only the useful frequency band of the antenna (e.g., 100–400 MHz for a 200 MHz antenna).  
This step improves the signal-to-noise ratio and sharpens reflections.

### 6. Migration
Repositions dipping reflections and diffractions to their true subsurface locations.  
Migration is essential for obtaining geometrically accurate images, especially in complex settings.

### 7. Time-to-Depth Conversion
Converts the vertical axis from two-way travel time (ns) to depth (m) using an estimated/measured EM velocity or dielectric constant $\varepsilon_r$. For a horizontally layered medium use the appropriate average (e.g., root-mean-square for layered media along the ray path). A rule-of-thumb for typical soils:

| Material          | Relative permittivity (εᵣ) | Notes |
|-------------------|-----------------------------|--------|
| **Air**           | ≈ 1                         | Reference, velocity ≈ 0.30 m/ns |
| **Ice**           | ≈ 3.1–3.2                   | Depends on purity and temperature |
| **Pure water**    | ≈ 80                        | At 20 °C; strongly temperature-dependent |
| **Dry sand**      | ≈ 3–5                       | Low moisture content |
| **Moist sand**    | ≈ 6–12                      | Moderate moisture |
| **Clay / Organic soil** | ≈ 10–30              | High water and mineral content |
| **Concrete (dry)** | ≈ 4–8                      | Varies with porosity and composition |
| **Concrete (wet)** | ≈ 8–15                     | Increases significantly with moisture |


Then use $z = c_0 t_{2w}/(2\sqrt{\varepsilon_r})$.
  
This is the final step that enables geological or engineering interpretation.


> **Practical notes:**
> - GPR resolution is roughly $\lambda/4$ (vertical) for a dominant frequency component. 
> - Attenuation in conductive soils (high $\sigma$) strongly limits penetration and bandwidth.  
> - Antenna coupling (near-surface) and system response matter: recorded wavelets are not ideal Ricker pulses. 
> - Migration and deconvolution improve imaging (RTM and Kirchhoff migration the most common).  

---
### References & further reading

- Daniels, D.J., *Ground Penetrating Radar*, 2nd ed. (2004).  
- Annan, A.P., *Ground penetrating radar theory and applications* (2009).  
- Taflove & Hagness, *Computational Electrodynamics: The Finite-Difference Time-Domain Method* (2005).